In [0]:
!pip install --upgrade grpcio
!pip install tensorflow==2.0
!pip install tensorflow-gpu

In [1]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.experimental.list_physical_devices())
print(tf.test.is_gpu_available())
print(tf.test.is_built_with_cuda())

2.0.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
True
True


In [0]:
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import backend as K

In [0]:
%load_ext tensorboard

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

data_dir_path = 'gdrive/My Drive/tensorflow_datasets/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
train_validation_split = tfds.Split.TRAIN.subsplit([9, 1])
((train_dataset, validation_dataset), test_dataset), info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True, data_dir=data_dir_path, download=False, split=(train_validation_split, tfds.Split.TEST))

In [6]:
print("info features: ", info.features)
encoder = info.features["text"].encoder
print("\n Vocabulary size: ", encoder.vocab_size)

info features:  FeaturesDict({
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    'text': Text(shape=(None,), dtype=tf.int64, encoder=<SubwordTextEncoder vocab_size=8185>),
})

 Vocabulary size:  8185


In [0]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

# fills a buffer with buffer_size elements, then randomly samples elements from this buffer, replacing the selected elements with new elements. For perfect shuffling, a buffer size greater than or equal to the full size of the dataset is required.
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
# output_shapes returns the shape of each component of an element of this dataset.
train_dataset = train_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(train_dataset))

validation_dataset = validation_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(validation_dataset))

test_dataset = test_dataset.padded_batch(BATCH_SIZE, tf.compat.v1.data.get_output_shapes(test_dataset))

The base class [RNN](https://www.tensorflow.org/api_docs/python/tf/keras/layers/RNN?version=stable) for recurrent layers inherits from class keras.layers.Layer. Each RNN cell isntance must have the following:

*   state_size attribute
*   output_size attriute
*   call(input_at_t, state_at_t) method, which return output_at_t and state_at_t_plus_1.
*   get_initial_state(inputs=None, batch_size=None, dtype=None) method that creates a tensor meant to be fed to call() as the initial state, if the user didn't specify any initial state via other means.



In [0]:
class CustomRNNCell(tf.keras.layers.Layer):

    def __init__(self, units, **kwargs):
        self.units = units
        self.state_size = units
        super(CustomRNNCell, self).__init__(**kwargs)

    def build(self, input_shape):
        # add_weight is from base_layer class, used to add a new variable to the layer
        
        # kernel initializer, weight matrix used for the linear transformation of the inputs
        self.kernel = self.add_weight(shape=(input_shape[-1], self.units), initializer='uniform', name='kernel')

        # recurrent initializer, weight matrix used for the linear transformation of the recurrent state.
        self.recurrent_kernel = self.add_weight(shape=(self.units, self.units), initializer='uniform', name='recurrent_kernel')
        self.built = True

    def call(self, inputs, states):
        prev_output = states[0]
        h = K.dot(inputs, self.kernel)
        output = h + K.dot(prev_output, self.recurrent_kernel)
        # For a simple RNN, the output_at_t and hidden_state_at_t_plus_1 is same.
        return output, [output]

In [9]:
use_dropout = False

model = tf.keras.Sequential()

model.add(tf.keras.layers.Embedding(input_dim=encoder.vocab_size, output_dim=64))
model.add(tf.keras.layers.RNN([CustomRNNCell(8)]))
if use_dropout:
    model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
rnn (RNN)                    (None, 8)                 576       
_________________________________________________________________
dense (Dense)                (None, 16)                144       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 524,577
Trainable params: 524,577
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
def binary_crossentropy(y_true, y_pred, from_logits=False):
    y_true = tf.cast(y_true, y_pred.dtype)
    def get_epsilon():
        # epsilon_value = 1e-7
        return tf.keras.backend.epsilon()
    
    if not from_logits:
#             if y_pred.op.type == "Sigmoid":
#                 TODO: dont clip. Use the from_logits directly.
        epsilon = get_epsilon()
        clipped_y_pred = tf.clip_by_value(y_pred, clip_value_min=epsilon, clip_value_max=(1.-epsilon))
        bce = tf.math.multiply(y_true, tf.math.log(tf.math.add(clipped_y_pred, epsilon)))
        temp = tf.math.multiply(tf.math.subtract(1., y_true), tf.math.log(tf.math.add(epsilon, tf.math.subtract(1., clipped_y_pred))))
        return tf.math.negative(tf.reduce_mean(tf.math.add(bce, temp)))
    else:
        # - x * z + log(1 + exp(x)), x = logits, z = labels
        return tf.reduce_mean(tf.math.add(tf.math.negative(tf.math.multiply(y_pred, y_true)), tf.math.log(tf.math.add(1., tf.math.exp(y_pred)))))

In [0]:
model.compile(loss=binary_crossentropy, optimizer='adam', metrics=['accuracy'])

In [0]:
callbacks = [
  # Write TensorBoard logs to `./tf_logs/rnn` directory
  tf.keras.callbacks.TensorBoard(log_dir='gdrive/My Drive/Colab Notebooks/DLCodeSnippets/tf_logs/rnn_1', histogram_freq=5, write_images=True, write_graph=True),
]

In [0]:
history = model.fit(train_dataset, epochs=10, validation_data=validation_dataset, validation_steps=5, callbacks=callbacks)

Epoch 1/10
      1/Unknown - 4s 4s/step - loss: 0.6932 - accuracy: 0.4375WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (1.823969). Check your callbacks.


352/352 [==============================] - 400s 1s/step - loss: 0.6932 - accuracy: 0.4979 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
352/352 [==============================] - 408s 1s/step - loss: 0.6931 - accuracy: 0.4971 - val_loss: 0.6923 - val_accuracy: 0.5094
Epoch 3/10
352/352 [==============================] - 409s 1s/step - loss: 0.6931 - accuracy: 0.4918 - val_loss: 0.6926 - val_accuracy: 0.4969
Epoch 4/10
196/352 [===============>..............] - ETA: 3:02 - loss: 0.6925 - accuracy: 0.5016